In [ ]:
# set working dir
import os
os.chdir("../")

In [ ]:
from copy import deepcopy
from types import SimpleNamespace

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import scienceplots
plt.style.reload_library()
plt.style.use("science")
from matplotlib import ticker
import tol_colors as tc

from run import utils

In [ ]:
# init config
config = SimpleNamespace()
config.prob = "sp"
config.grid = (5, 5)
config.feat = 5
config.net = []
config.optm = "adam"
config.batch = 32
config.l1 = 0.0
config.l2 = 0.0
config.rel = False
config.lan = "gurobi"
config.proc = 8
config.path = "./res"
config.sftp = False

In [ ]:
# polynomial degree
degs = [2, 6]

In [ ]:
def getPath(config, mthd):
    config = deepcopy(config)
    if mthd == "spo":
        config.mthd = "spo"
        config.lr = 1e-2
    if mthd == "spo l1":
        config.mthd = "spo"
        config.batch = 32
        config.lr = 1e-2
        config.l1 = 1e-2
    if mthd == "spo l2":
        config.mthd = "spo"
        config.lr = 1e-2
        config.l2 = 1e-2
    if mthd == "dbb":
        config.mthd = "dbb"
        config.loss = "r"
        config.lr = 1e-1
        config.smth = 20
    if mthd == "dbb l1":
        config.mthd = "dbb"
        config.loss = "r"
        config.lr = 1e-1
        config.smth = 20
        config.l1 = 1e-2
    if mthd == "dbb l2":
        config.mthd = "dbb"
        config.loss = "r"
        config.lr = 1e-1
        config.smth = 20
        config.l2 = 1e-2
    if mthd == "dpo":
        config.mthd = "dpo"
        config.lr = 1e-2
        config.samp = 1
        config.eps = 1.0
    if mthd == "dpo l1":
        config.mthd = "dpo"
        config.lr = 1e-2
        config.samp = 1
        config.eps = 1.0
        config.l1 = 1e-2
    if mthd == "dpo l2":
        config.mthd = "dpo"
        config.lr = 1e-2
        config.samp = 1
        config.eps = 1.0
        config.l2 = 1e-2
    if mthd == "pfyl":
        config.mthd = "pfyl"
        config.lr = 1e-2
        config.samp = 1
        config.eps = 1.0
    if mthd == "pfyl l1":
        config.mthd = "pfyl"
        config.lr = 1e-2
        config.samp = 1
        config.eps = 1.0
        config.l1 = 1e-2
    if mthd == "pfyl l2":
        config.mthd = "pfyl"
        config.lr = 1e-2
        config.samp = 1
        config.eps = 1.0
        config.l2 = 1e-2
    path = utils.getSavePath(config)
    return path

In [ ]:
def getDf(config, degs, mthd):
    dfs = pd.DataFrame()
    for deg in degs:
        config.deg = deg
        df = pd.read_csv(getPath(config, mthd))
        dfs[deg] = df["Unamb SPO"]
    return dfs

In [ ]:
def lighten(color, amount=0.9):
    """
    Lightens the given color by multiplying (1-luminosity) by the given amount.
    """
    import matplotlib.colors as mc
    import colorsys
    try:
        c = mc.cnames[color]
    except:
        c = color
    c = np.array(colorsys.rgb_to_hls(*mc.to_rgb(c)))
    return colorsys.hls_to_rgb(c[0],1-amount * (1-c[1]),c[2])

In [ ]:
def drawPlot(config, data, noise):
    # set config
    config = deepcopy(config)
    config.data = data
    config.noise = noise
    # color map
    cmap = tc.tol_cmap("rainbow_discrete")(np.linspace(0, 1, 22))
    colors = [cmap[16], cmap[7], cmap[10], cmap[5]]
    for i in range(len(colors)):
        colors[i] = lighten(colors[i])
    # get df
    df_spo   = getDf(config, degs, "spo")
    df_spo1  = getDf(config, degs, "spo l1")
    df_spo2  = getDf(config, degs, "spo l2")
    df_dbb   = getDf(config, degs, "dbb")
    df_dbb1  = getDf(config, degs, "dbb l1")
    df_dbb2  = getDf(config, degs, "dbb l2")
    df_dpo   = getDf(config, degs, "dpo")
    df_dpo1  = getDf(config, degs, "dpo l1")
    df_dpo2  = getDf(config, degs, "dpo l2")
    df_pfyl  = getDf(config, degs, "pfyl")
    df_pfyl1 = getDf(config, degs, "pfyl l1")
    df_pfyl2 = getDf(config, degs, "pfyl l2")
    # draw boxplot
    fig = plt.figure(figsize=(16,6))
    #######################################################################################################################
    c = colors[0]
    bp1 = plt.boxplot(df_spo, boxprops=dict(facecolor=c, color=c, linewidth=4), medianprops=dict(color="w", linewidth=2),
                      whiskerprops=dict(color=c, linewidth=2), capprops=dict(color=c, linewidth=2),
                      flierprops=dict(markeredgecolor=c, marker="o", markersize=5, markeredgewidth=2),
                      patch_artist=True, positions=np.arange(df_spo.shape[1])-0.44, widths=0.06)
    c = colors[0]
    bp2 = plt.boxplot(df_spo1, boxprops=dict(facecolor=c, color=c, linewidth=4), medianprops=dict(color="w", linewidth=2),
                      whiskerprops=dict(color=c, linewidth=2), capprops=dict(color=c, linewidth=2),
                      flierprops=dict(markeredgecolor=c, marker="o", markersize=5, markeredgewidth=2),
                      patch_artist=True, positions=np.arange(df_spo.shape[1])-0.36, widths=0.06)
    for box in bp2['boxes']:
        box.set(hatch="++++", fill=False)
    c = colors[0]
    bp3 = plt.boxplot(df_spo2, boxprops=dict(facecolor=c, color=c, linewidth=4), medianprops=dict(color="w", linewidth=2),
                      whiskerprops=dict(color=c, linewidth=2), capprops=dict(color=c, linewidth=2),
                      flierprops=dict(markeredgecolor=c, marker="o", markersize=5, markeredgewidth=2),
                      patch_artist=True, positions=np.arange(df_spo.shape[1])-0.28, widths=0.06)
    for box in bp3["boxes"]:
        box.set(hatch="OO", fill=False)
    #######################################################################################################################
    c = colors[1]
    bp4 = plt.boxplot(df_dbb, boxprops=dict(facecolor=c, color=c, linewidth=4), medianprops=dict(color="w", linewidth=2),
                      whiskerprops=dict(color=c, linewidth=2), capprops=dict(color=c, linewidth=2),
                      flierprops=dict(markeredgecolor=c, marker="o", markersize=5, markeredgewidth=2),
                      patch_artist=True, positions=np.arange(df_dbb.shape[1])-0.20, widths=0.06)
    c = colors[1]
    bp5 = plt.boxplot(df_dbb1, boxprops=dict(facecolor=c, color=c, linewidth=4), medianprops=dict(color="w", linewidth=2),
                      whiskerprops=dict(color=c, linewidth=2), capprops=dict(color=c, linewidth=2),
                      flierprops=dict(markeredgecolor=c, marker="o", markersize=5, markeredgewidth=2),
                      patch_artist=True, positions=np.arange(df_dbb.shape[1])-0.12, widths=0.06)
    for box in bp5["boxes"]:
        box.set(hatch="++++", fill=False)
    c = colors[1]
    bp6 = plt.boxplot(df_dbb2, boxprops=dict(facecolor=c, color=c, linewidth=4), medianprops=dict(color="w", linewidth=2),
                      whiskerprops=dict(color=c, linewidth=2), capprops=dict(color=c, linewidth=2),
                      flierprops=dict(markeredgecolor=c, marker="o", markersize=5, markeredgewidth=2),
                      patch_artist=True, positions=np.arange(df_dbb.shape[1])-0.04, widths=0.06)
    for box in bp6["boxes"]:
        box.set(hatch="OO", fill=False)
    #######################################################################################################################
    c = colors[2]
    bp7 = plt.boxplot(df_dpo, boxprops=dict(facecolor=c, color=c, linewidth=4), medianprops=dict(color="w", linewidth=2),
                      whiskerprops=dict(color=c, linewidth=2), capprops=dict(color=c, linewidth=2),
                      flierprops=dict(markeredgecolor=c, marker="o", markersize=5, markeredgewidth=2),
                      patch_artist=True, positions=np.arange(df_dbb.shape[1])+0.04, widths=0.06)
    c = colors[2]
    bp8 = plt.boxplot(df_dpo1, boxprops=dict(facecolor=c, color=c, linewidth=4), medianprops=dict(color="w", linewidth=2),
                      whiskerprops=dict(color=c, linewidth=2), capprops=dict(color=c, linewidth=2),
                      flierprops=dict(markeredgecolor=c, marker="o", markersize=5, markeredgewidth=2),
                      patch_artist=True, positions=np.arange(df_dbb.shape[1])+0.12, widths=0.06)
    for box in bp8["boxes"]:
        box.set(hatch="++++", fill=False)
    c = colors[2]
    bp9 = plt.boxplot(df_dpo2, boxprops=dict(facecolor=c, color=c, linewidth=4), medianprops=dict(color="w", linewidth=2),
                      whiskerprops=dict(color=c, linewidth=2), capprops=dict(color=c, linewidth=2),
                      flierprops=dict(markeredgecolor=c, marker="o", markersize=5, markeredgewidth=2),
                      patch_artist=True, positions=np.arange(df_dbb.shape[1])+0.20, widths=0.06)
    for box in bp9["boxes"]:
        box.set(hatch="OO", fill=False)
    #######################################################################################################################
    c = colors[3]
    bpa = plt.boxplot(df_pfyl, boxprops=dict(facecolor=c, color=c, linewidth=4), medianprops=dict(color="w", linewidth=2),
                      whiskerprops=dict(color=c, linewidth=2), capprops=dict(color=c, linewidth=2),
                      flierprops=dict(markeredgecolor=c, marker="o", markersize=5, markeredgewidth=2),
                      patch_artist=True, positions=np.arange(df_dbb.shape[1])+0.28, widths=0.06)
    c = colors[3]
    bpb = plt.boxplot(df_pfyl1, boxprops=dict(facecolor=c, color=c, linewidth=4), medianprops=dict(color="w", linewidth=2),
                      whiskerprops=dict(color=c, linewidth=2), capprops=dict(color=c, linewidth=2),
                      flierprops=dict(markeredgecolor=c, marker="o", markersize=5, markeredgewidth=2),
                      patch_artist=True, positions=np.arange(df_dbb.shape[1])+0.36, widths=0.06)
    for box in bpb["boxes"]:
        box.set(hatch="++++", fill=False)
    c = colors[3]
    bpc = plt.boxplot(df_pfyl2, boxprops=dict(facecolor=c, color=c, linewidth=4), medianprops=dict(color="w", linewidth=2),
                      whiskerprops=dict(color=c, linewidth=2), capprops=dict(color=c, linewidth=2),
                      flierprops=dict(markeredgecolor=c, marker="o", markersize=5, markeredgewidth=2),
                      patch_artist=True, positions=np.arange(df_dbb.shape[1])+0.44, widths=0.06)
    for box in bpc["boxes"]:
        box.set(hatch="OO", fill=False)
    # vertical line
    plt.axvline(x=0.5, color="k", linestyle="--", linewidth=1.5)
    #plt.axvline(x=1.5, color="k", linestyle="--", linewidth=1.5)
    #plt.axvline(x=2.5, color="k", linestyle="--", linewidth=1.5)
    # grid
    plt.grid(color="grey", alpha=0.5, linewidth=0.5, which="major", axis="y")
    # labels and ticks
    plt.xlabel("Polynomial Degree", fontsize=36)
    plt.xticks(ticks=range(len(degs)), labels=degs, fontsize=28)
    plt.ylabel("Normalized Regret", fontsize=36)
    plt.yticks(fontsize=24)
    plt.xlim(-0.5, 1.5)
    plt.ylim(-0.01, 0.39)
    ax = plt.gca()
    ax.yaxis.set_major_formatter(ticker.FormatStrFormatter("%.2f"))
    plt.title("Shortest Path\nTraining Set Size = {}, Noise Half−width = {}".format(config.data, config.noise), fontsize=30)
    plt.legend([bp1["boxes"][0], bp2["boxes"][0], bp3["boxes"][0], bp4["boxes"][0], bp5["boxes"][0], bp6["boxes"][0],
                bp7["boxes"][0], bp8["boxes"][0], bp9["boxes"][0], bpa["boxes"][0], bpb["boxes"][0], bpc["boxes"][0]],
               ["SPO+","SPO+ L1","SPO+ L2","DBB","DBB L1","DBB L2","DPO","DPO L1","DPO L2","PFYL","PFYL L1","PFYL L2"], 
               fontsize=20, loc=2, labelspacing=0.2,
               handlelength=1, ncol=4)
    # save
    plt.show()
    fig.savefig("./images/reg-sp-n{}e{}.pdf".format(data,int(10*noise)), dpi=300)

## n = 100, e = 0

In [ ]:
drawPlot(config, data=100, noise=0.0)

## n = 100, e = 0.5 

In [ ]:
drawPlot(config, data=100, noise=0.5)

## n = 1000, e = 0

In [ ]:
drawPlot(config, data=1000, noise=0.0)

## n = 1000, e = 0.5 

In [ ]:
drawPlot(config, data=1000, noise=0.5)

## n = 5000, e = 0

In [ ]:
drawPlot(config, data=5000, noise=0.0)

## n = 5000, e = 0.5 

In [ ]:
drawPlot(config, data=5000, noise=0.5)